# WORK IN PROGRESS, TUTORIAL NOT COMPLETE

# Link Prediction on embeddings
The link prediction problem consists in trying to predict the edges that actually exists in the graph.

In our implementation we can execute training on big graphs lazily.

In [1]:
from embiggen import GraphTransformer
from ensmallen_graph import EnsmallenGraph

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import silence_tensorflow.auto
import numpy as np
from tensorflow.distribute import MirroredStrategy
from plot_keras_history import plot_history
from plot_keras_history.utils import chain_histories
import os
import pandas as pd
from glob import glob

## Loading the graphs
We load the ppi graph as a weighted undirected graph.

In [2]:
graph = EnsmallenGraph.from_csv(
    edge_path="../data/ppi/edges.tsv",
    sources_column="subject",
    destinations_column="object",
    directed=False,
    weights_column="weight"
)

As first thing, we plot a short report showing all the avalable graph details, including the number of edges, nodes, trap nodes and both the connected components and the strongly connected components.

In [3]:
graph.report()

{'degrees_mode': '1',
 'is_directed': 'false',
 'unique_node_types_number': '0',
 'density': '0.001993564869255138',
 'strongly_connected_components_number': '181',
 'connected_components_number': '181',
 'unique_edge_types_number': '0',
 'traps_rate': '0',
 'singleton_nodes': '0',
 'selfloops_percentage': '0',
 'degrees_mean': '34.25941227814955',
 'bidirectional_percentage': '1',
 'degrees_median': '11',
 'nodes_number': '17185',
 'edges_number': '588748'}

## Creating the holdouts
For every one of these examples, we are going to use two experimental setups.

We split the graph into a training and validation components.

Particularly, the training partition will have the same number of connected components as the original graph, while the validation may have more.

We split the two datasets at $0.8$, as done in [Yue et al.](https://academic.oup.com/bioinformatics/article/36/4/1241/5581350).

We are going to us two different experimental setups:

- One, based on the [Yue et al.](https://academic.oup.com/bioinformatics/article/36/4/1241/5581350) and [Leskovec et al.](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5108654/) experimental setups, where we completely separate training negatives from validation negatives and leave the validation parts of the graphs (both positives and negatives) as "holes" in the graph, neither negative nor positive. This may be slighly sconnected from the real scenario, where we cannot remove from a graph part of the validation edges since they are not known. In particular, the negatives edges used in the validation have the same cardinality as the validation positive edges.

**For the context of CBOW or GloVe, both setups are the one and the same. The difference is relevant in SkipGram embedding and LinkPrediction.**

### Splitting the graph into the two components

In [4]:
training, validation = graph.connected_holdout(42, 0.8)

### Preparing the experimental setup
For the experimental setup, we are using walk lengths of $100$ nodes, with batch size (number of walks to be considered in each iteration) of $2^{15} = 32768$.

We are going to use 20 walk iterations for each node.

We are going to use a window size of $4$, meaning $4$ nodes on the left and right of every central node. 

Consider that the first *window_size* values on the left and the right of the walks will be trimmed.

To generate the walks we will use the parameters $p$ and $q$ equal to $1.0$.

The *embedding_size* is $100$. For the glove loss, we are going to use an alpha of $0.75$.

We are going to use Nadam as obtimizer. We are going to use an Early Stopping criterion on the *validation loss*, with patience $5$ and delta $0.0001$.

The model will be trained up to $1000$ epochs.

#### Setting up the parameters

In [6]:
batch_size=2**15
embedding_size=100
iterations=20
window_size=4
p=1.0
q=1.0
patience=5
delta=0.0001
batches_per_epoch = 2**8

## Defining the neural network for Link Prediction

In [ ]:
def get_link_prediction_model(embedding_size:int):
    model = Sequential([
        Inp
    ])

## Prepare experimental setup for Leskovec

## Prepare new experimental setup